# Compare models

Two models (PaDiM and PatchCore) will be trained and evaluated with a per-image metric (`AULogPImO`).

See the notebook `502a_perimg_metrics.ipynb` to firs get familiarized with this metric.

The model performances are exported and loaded back, then the two models are compared based on each image's scores.  

In [ ]:
# TODO add link to notebook mentioned above
# TODO pick model params to make them more competitive

# Installing Anomalib

The easiest way to install anomalib is to use pip. You can install it from the command line using the following command:


In [ ]:
%pip install anomalib

In [ ]:
%load_ext autoreload

In [ ]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# Data

We will use MVTec AD DataModule. 

> See [these notebooks](https://github.com/openvinotoolkit/anomalib/tree/main/notebooks/100_datamodules) for more details on datamodules. 

We assume that `datasets` directory is created in the `anomalib` root directory and `MVTec` dataset is located in `datasets` directory.

In [ ]:
from pathlib import Path

# NOTE: Provide the path to the dataset root directory.
#   If the datasets is not downloaded, it will be downloaded to this directory.
dataset_root = Path.cwd().parent.parent.parent / "datasets" / "MVTec"

We will be working on a segmentation task. 

In [ ]:
from anomalib.data import TaskType

task = TaskType.SEGMENTATION

And with the `hazelnut` category at resolution of 256x256 pixels.

In [ ]:
from anomalib.data.mvtec import MVTec

datamodule = MVTec(
    root=dataset_root,
    category="hazelnut",
    image_size=256,
    train_batch_size=32,
    eval_batch_size=32,
    num_workers=8,
    task=task,
)
datamodule.setup()
i, data = next(enumerate(datamodule.test_dataloader()))
print(f'Image Shape: {data["image"].shape} Mask Shape: {data["mask"].shape}')

# Models

We train two models: PaDiM and PatchCore.

> See [these notebooks](https://github.com/openvinotoolkit/anomalib/tree/main/notebooks/200_models) for more details on models. 

## PaDiM

The next cell instantiates and trains PaDiM.

The `MetricsConfigurationCallback()` will not have metric because they will be created manually.

In [ ]:
from pytorch_lightning import Trainer

from anomalib.utils.callbacks import MetricsConfigurationCallback, PostProcessingConfigurationCallback
from anomalib.post_processing import NormalizationMethod, ThresholdMethod
from anomalib.models import Padim

padim = Padim(
    input_size=(256, 256),
    layers=[
        "layer1",
        "layer2",
    ],
    backbone="resnet18",
    pre_trained=True,
)

trainer = Trainer(
    callbacks=[
        PostProcessingConfigurationCallback(
            normalization_method=NormalizationMethod.MIN_MAX,
            threshold_method=ThresholdMethod.ADAPTIVE,
        ),
        MetricsConfigurationCallback(),
    ],
    max_epochs=1,
    num_sanity_val_steps=0,  # does not work for padim
    accelerator="auto",
)

trainer.fit(datamodule=datamodule, model=padim)

## PatchCore

The next cell instantiates and trains PatchCore.

The `MetricsConfigurationCallback()` will not have metric because they will be created manually.

In [ ]:
%autoreload 2

from anomalib.post_processing import ThresholdMethod

from pytorch_lightning import Trainer
from anomalib.models import Patchcore

patchcore = Patchcore(
    input_size=(256, 256),
    backbone="resnet18",
    layers=[
        "layer3",
    ],
    coreset_sampling_ratio=0.1,
    num_neighbors=9,
    pre_trained=True,
)

trainer = Trainer(
    callbacks=[
        PostProcessingConfigurationCallback(
            normalization_method=NormalizationMethod.MIN_MAX,
            threshold_method=ThresholdMethod.ADAPTIVE,
        ),
        MetricsConfigurationCallback(),
    ],
    max_epochs=1,
    num_sanity_val_steps=0,  # does not work for patchcore
    accelerator="auto",
)

trainer.fit(datamodule=datamodule, model=patchcore)

# Process test images

This part is usually happening automatically but here we want to extract the outputs manually.

In [ ]:
import torch

_ = padim.eval()
_ = patchcore.eval()

anomaly_maps_padim = []
anomaly_maps_patchcore = []
masks = []

for batchidx, batch in enumerate(datamodule.test_dataloader()):
    anomaly_maps_padim.append(padim.test_step_end(padim.test_step(batch, batchidx))["anomaly_maps"].squeeze(1))
    anomaly_maps_patchcore.append(
        patchcore.test_step_end(patchcore.test_step(batch, batchidx))["anomaly_maps"].squeeze(1)
    )
    masks.append(padim.test_step_end(padim.test_step(batch, batchidx))["mask"].int())

anomaly_maps_padim = torch.cat(anomaly_maps_padim)
anomaly_maps_patchcore = torch.cat(anomaly_maps_patchcore)
masks = torch.cat(masks)

print(f"{anomaly_maps_padim.shape=} {anomaly_maps_patchcore.shape=} {masks.shape=}")

# `AULogPImO`

> Area Under the Log Per-Image Overlap curve; pronounced "a-u-log-pee-mo"

Reminder of the metric definition.

The `LogPImO` curves have a shared X-axis and a per-image Y-axis.

The X-axis ("shared-FPR"):
- is a metric of False Positives only in the normal images (here it is the log10 of the mean of in-image FPRs, which equals the set-FPR)
- is shared by all anomalous images/curves

The Y-axis: 
- is, at a given threshold (index by the X-axis), the **overlap** between the binary predicted mask and the ground truth mask, which corresponds to the in-image TPR
- has one value per image, so there is one LogPImO curve per image. 

> - FPR: False Positive Rate
> - TPR: True Positive Rate

The `AULogPImO` $\in [0, 1]$ is the area under each of these curves normalized by the maximum possible area (when TPR is constant at 1 for all FPR values):

$$
    \frac{\int_{L}^{U} \; \text{TPR}( \, \text{FPR} \, ) \; d\text{log(FPR)}}{log(U/L)} 
    \quad ,
    
$$

where $L \in (0, 1)$ is the shared-FPR lower bound, and $U \in (0, 1]$ is the shared-FPR upper bound such that $U > L$.

> **Score of a random model**
> 
> The `AULogPImO` score of a random model (i.e. $\text{TPR}(\text{FPR}) = \text{FPR}$) is 
>    
> $$
> \frac{U - L}{log(U / L)}
> \quad .
> $$

## PaDiM

In [ ]:
%autoreload 2
import scipy as sp
from anomalib.utils.metrics.perimg import AULogPImO

aulogpimo_padim = AULogPImO(lbound=0.001, ubound=0.03)
print(f"AULogPImO of a random model: {aulogpimo_padim.random_model_auc:.1%}")
aulogpimo_padim.cpu()
aulogpimo_padim.update(anomaly_maps_padim, masks)
pimoresult_padim, aucs_padim = aulogpimo_padim.compute()
print(sp.stats.describe(aucs_padim[~torch.isnan(aucs_padim)]))  # `~torch.isnan(aucs)` is removing the `nan`s

In [ ]:
fig, axes = aulogpimo_padim.plot()
_ = fig.suptitle("AULogPImO of PADIM")

## PatchCore

In [ ]:
%autoreload 2
import scipy as sp
from anomalib.utils.metrics.perimg import AULogPImO

aulogpimo_patchcore = AULogPImO(lbound=0.001, ubound=0.03)
print(f"AULogPImO of a random model: {aulogpimo_patchcore.random_model_auc:.1%}")
aulogpimo_patchcore.cpu()
aulogpimo_patchcore.update(anomaly_maps_patchcore, masks)
pimoresult_patchcore, aucs_patchcore = aulogpimo_patchcore.compute()
print(sp.stats.describe(aucs_patchcore[~torch.isnan(aucs_patchcore)]))  # `~torch.isnan(aucs)` is removing the `nan`s

In [ ]:
fig, axes = aulogpimo_patchcore.plot()
_ = fig.suptitle("AULogPImO of PatchCore")

## Save

Save the AUCs from both models in the disk.


In [ ]:
from pathlib import Path

CACHE = Path() / ".cache" / "502b"
CACHE.mkdir(exist_ok=True, parents=True)

In [ ]:
aulogpimo_padim.save(CACHE / "aulogpimo_padim.json")
aulogpimo_patchcore.save(CACHE / "aulogpimo_patchcore.json")

%ls -lh $CACHE

In [ ]:
del aulogpimo_patchcore, pimoresult_patchcore, aucs_patchcore
del aulogpimo_padim, pimoresult_padim, aucs_padim

# Load

Reload back from the disk

In [ ]:
from anomalib.utils.metrics.perimg import AULogPImO

pimoresult_padim, aucs_padim = AULogPImO.load(CACHE / "aulogpimo_padim.json")
lbound, ubound = aucs_padim["lbound"], aucs_padim["ubound"]  # they are the same for both models
aucs_padim = aucs_padim["aulogpimo"]

pimoresult_patchcore, aucs_patchcore = AULogPImO.load(CACHE / "aulogpimo_patchcore.json")
aucs_patchcore = aucs_patchcore["aulogpimo"]

In [ ]:
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt

# the funcional interface of the plotting funcions have to be used because the
# aulogpimo object is gone
AULogPImO
from anomalib.utils.metrics.perimg.pimo import AULogPImO
from anomalib.utils.metrics.perimg.plot import plot_boxplot_logpimo_curves, plot_aulogpimo_boxplot
from anomalib.utils.metrics.perimg.common import perimg_boxplot_stats

fig, axes = plt.subplots(2, 2, figsize=(14, 13))

for axrow, pimoresult, aucs, model in zip(
    axes, [pimoresult_padim, pimoresult_patchcore], [aucs_padim, aucs_patchcore], ["PaDiM", "PatchCore"]
):
    _ = plot_aulogpimo_boxplot(
        aucs,
        pimoresult.image_classes,
        # optional
        random_model_auc=AULogPImO.random_model_auc_from_bounds(lbound, ubound),
        ax=axrow[0],
    )
    _ = axrow[0].set_title(f"{model} - boxplot")

    bp_stats = perimg_boxplot_stats(aucs, pimoresult.image_classes, only_class=1)

    _ = plot_boxplot_logpimo_curves(
        pimoresult.shared_fpr,
        pimoresult.tprs,
        pimoresult.image_classes,
        bp_stats,
        lbound,
        ubound,
        ax=axrow[1],
    )
    _ = axrow[1].set_title(f"{model} - curves")

minmin = min([ax.get_xlim()[0] for ax in axes[:, 0].flatten()])
maxmax = max([ax.get_xlim()[1] for ax in axes[:, 0].flatten()])
for ax in axes[:, 0].flatten():
    _ = ax.set_xlim(minmin, maxmax)

fig.suptitle("AULogPImO: PaDiM vs PatchCore", fontsize=20)

# Image by image comparison (`PaDiM` vs `PatchCore`)

## Parametric (metric comparison)

Here we compare models directly with the value of `AULogPImO`. 

The plot shows the metric value of each image on the Y-axis, and the horizontal bars correspond to their average.

The table shows the confidence level that `model1` is better than `model2` based on the average score.

> The confidence level is `1 - pvalue` from a paired T-test with null hypothesis that `mean(model1) == mean(model2)` and alternative hypothesis that `mean(model1) > mean(model2)`.
>
> Higher values indicate more confidence that the null hypothesis should be rejected and `mean(model1) > mean(model2)` indeed.
>
> See [`scipy.stats.ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) and ["Dependent t-test for paired samples" at Wikipedia's page on "Student's t-test"](https://en.wikipedia.org/wiki/Student's_t-test#Dependent_t-test_for_paired_samples).

In [ ]:
from anomalib.utils.metrics.perimg.pimo import AULogPImO
from anomalib.utils.metrics.perimg.plot import compare_models_perimg
from anomalib.utils.metrics.perimg.common import compare_models_parametric

models = {
    "PaDiM": aucs_padim,
    "PatchCore": aucs_patchcore,
}

compare_models_perimg(
    models,
    "AULogPImO",
    random_model_score=AULogPImO.random_model_auc_from_bounds(lbound, ubound),
)

compare_models_parametric(models)

## Non-parametric (rank comparison)

TODO describe

In [ ]:
from anomalib.utils.metrics.perimg.pimo import AULogPImO
from anomalib.utils.metrics.perimg.plot import compare_models_perimg_rank
from anomalib.utils.metrics.perimg.common import compare_models_nonparametric

models = {
    "PaDiM": aucs_padim,
    "PatchCore": aucs_patchcore,
}

compare_models_perimg_rank(
    models,
    "AULogPImO",
)

compare_models_nonparametric(models)

# Adding a new model

Now we execute the same train-eval process from before using FastFlow.

Then, we will redo the same comparisons above but with all the three models at the same time.

# FastFlow

Simplified from [`notebooks/200_models/201_fastflow.ipynb`](https://github.com/openvinotoolkit/anomalib/blob/main/notebooks/200_models/201_fastflow.ipynb).

In [ ]:
%autoreload 2

from functools import partial, update_wrapper
from types import MethodType
from anomalib.post_processing import ThresholdMethod
from pytorch_lightning import Trainer
from anomalib.models import Fastflow
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning import LightningModule, Trainer
from torch.optim import Optimizer, Adam

fastflow = Fastflow(
    input_size=(256, 256),
    backbone="resnet18",
    pre_trained=True,
    flow_steps=8,
    hidden_ratio=1.0,
    conv3x3_only=True,
)


def configure_optimizers(lightning_module: LightningModule, optimizer: Optimizer):
    """Override to customize the LightningModule.configure_optimizers` method."""
    return optimizer


fn = partial(
    configure_optimizers, optimizer=Adam(params=fastflow.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-5)
)
update_wrapper(fn, configure_optimizers)  # necessary for `is_overridden`
fastflow.configure_optimizers = MethodType(fn, fastflow)

trainer = Trainer(
    callbacks=[
        PostProcessingConfigurationCallback(
            normalization_method=NormalizationMethod.MIN_MAX,
            threshold_method=ThresholdMethod.ADAPTIVE,
        ),
        MetricsConfigurationCallback(
            task=TaskType.SEGMENTATION,
            pixel_metrics=[
                "AUROC",
            ],
        ),
        ModelCheckpoint(
            mode="max",
            monitor="pixel_AUROC",
        ),
        EarlyStopping(
            monitor="pixel_AUROC",
            mode="max",
            patience=3,
        ),
    ],
    accelerator="auto",
    devices=1,
    max_epochs=20,
    num_sanity_val_steps=0,
)
trainer.fit(datamodule=datamodule, model=fastflow)

In [ ]:
%autoreload 2
import scipy as sp
from anomalib.utils.metrics.perimg import AULogPImO
import torch

_ = fastflow.eval()

anomaly_maps_fastflow = []

for batchidx, batch in enumerate(datamodule.test_dataloader()):
    anomaly_maps_fastflow.append(fastflow.test_step_end(fastflow.test_step(batch, batchidx))["anomaly_maps"].squeeze(1))

anomaly_maps_fastflow = torch.cat(anomaly_maps_fastflow)
print(f"{anomaly_maps_fastflow.shape=}")

aulogpimo_fastflow = AULogPImO(lbound=0.001, ubound=0.03)
aulogpimo_fastflow.cpu()
aulogpimo_fastflow.update(anomaly_maps_fastflow, masks)

fig, axes = aulogpimo_fastflow.plot()
_ = fig.suptitle("AULogPImO of FastFlow")

_ = aulogpimo_fastflow.save(CACHE / "aulogpimo_fastflow.json")
%ls -lh $CACHE

pimoresult_fastflow, aucs_fastflow = AULogPImO.load(CACHE / "aulogpimo_fastflow.json")
aucs_fastflow = aucs_fastflow["aulogpimo"]

# Image by image comparison (all vs all)

The statistical tests are done pairwise.

Each cell in the tables show the confidence that `model1` (row) is better than `model2` (column).

In [ ]:
from matplotlib import pyplot as plt
from anomalib.utils.metrics.perimg.pimo import AULogPImO
from anomalib.utils.metrics.perimg.plot import compare_models_perimg_rank, compare_models_perimg
from anomalib.utils.metrics.perimg.common import compare_models_nonparametric, compare_models_parametric

models = {
    "PaDiM": aucs_padim,
    "PatchCore": aucs_patchcore,
    "FastFlow": aucs_fastflow,
}

fig, axes = plt.subplots(2, 1, figsize=(14, 13))

_ = compare_models_perimg(models, "AULogPImO", ax=axes[0])
_ = compare_models_perimg_rank(models, "AULogPImO", ax=axes[1])
fig.suptitle("AULogPImO: PaDiM vs PatchCore vs FastFlow", fontsize=20)

print("Parametric tests:")
compare_models_parametric(models)

print("Non-parametric tests:")
compare_models_nonparametric(models)

# Comparing to the benchmark

coming soon